In [1]:
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd

In [53]:
# webpage information
base_url = 'https://myanimelist.net/recommendations.php?s=recentrecs&t=anime&show='
pages = 20    # sebenarnya pagesnya ada banyak banget tapi cukupkan 20 dulu

In [54]:
# array to store information
anime_base = []
anime_recommendation = []
anime_reason = []
anime_power = []

In [55]:
# the script to capture data for each pages
for i in range(pages) :
    temp_url = base_url + str(i*100)
    
    # connect and download html webpage
    uClient = uReq(temp_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    
    # search for base anime and recommended anime rows
    animes = page_soup.findAll("div", {"class": "spaceit borderClass"})

    # select the title for anime base and recommendation and the reason, then put into given arrays
    for i in range(len(animes)) :
        anime_in_row = animes[i].table.tr.findAll("td")
        base_container = anime_in_row[0]
        recommendation_container = anime_in_row[1]

        title_base = base_container.findAll("a")[1]["title"]
        title_recommendation = recommendation_container.findAll("a")[1].strong.text
        reason = animes[i].findAll("div",{"class": "spaceit recommendations-user-recs-text"})[0].text.strip()

        # put the information to arrays
        sudah_ada = True
        if title_base not in anime_base :
            anime_base.append(title_base)
            anime_recommendation.append([])
            anime_reason.append([])
            anime_power.append([])
        idx = anime_base.index(title_base)
        if title_recommendation not in anime_recommendation[idx] :
            anime_recommendation[idx].append(title_recommendation)
            anime_reason[idx].append(reason)
            anime_power[idx].append(1)
            sudah_ada = False
        if (sudah_ada) :
            idx_rec = anime_recommendation[idx].index(title_recommendation)
            anime_power[idx][idx_rec] += 1

In [73]:
# save to csv
df = pd.DataFrame({
    'Anime' : anime_base,
    'Recommendation': anime_recommendation,
    'Reasoning': anime_reason,
    'Recommendation Power': anime_power
})

df.to_csv("Anime Recommendation.csv", index = False)